We'll use the python library gensim: https://radimrehurek.com/gensim/

In [2]:
import gensim


C:\Users\heidi\Anaconda2\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Load a premade word2vec model built on Google News articles.

Download from: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

In [3]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin', binary=True, limit=500000)

We limit the vocabulary to the 500,000 most common words.  Even at 500,000, it starts to get to nonsense words.  Here are the top 50 and bottom 50 words by frequency. And even for the real words that are infrequent, if a word is too obscure, it wouldn't make for a good clue.

In [3]:
print("Most common:",model.index2word[:50])
print("Least common:",model.index2word[-50:])

('Most common:', [u'</s>', u'in', u'for', u'that', u'is', u'on', u'##', u'The', u'with', u'said', u'was', u'the', u'at', u'not', u'as', u'it', u'be', u'from', u'by', u'are', u'I', u'have', u'he', u'will', u'has', u'####', u'his', u'an', u'this', u'or', u'their', u'who', u'they', u'but', u'$', u'had', u'year', u'were', u'we', u'more', u'###', u'up', u'been', u'you', u'its', u'one', u'about', u'would', u'which', u'out'])
('Least common:', [u'ideological_affinity', u'Rashtriya_Rifles_RR', u'Pedrotti', u'Frysinger', u'Ralph_Sacco', u'Ryan_Nece', u'Homs_Syria', u'BACK_TO_BACK', u'Nag_Hammadi', u'Dashan', u'Murape', u'Majolica', u'Sundvold', u'Jerryd', u'administered_subcutaneously', u'Pierre_Luc_Gagnon', u'Fedrizzi', u'CD_ROMS', u'Raynham_Mass.', u'NN_Vohra', u'Barraba', u'Delta_Upsilon', u'Roilo_Golez', u'Cindy_Scroggins', u'Iter', u'Ford_Expeditions', u'La_Toussuire', u'Hooksett_NH', u'ITCTransmission', u'wakeskate', u'Fervor', u'SAFT', u'steam_boiler', u'Moskwa', u'Inet_electronic', u'2A

Here's an example Codenames board.  `blue` is one team's words, `red` the other and `assassin` is the assassin word.

In [8]:
# board = {
#     'blue': ['ambulance', 'hospital', 'spell', 'lock', 'charge', 'tail', 'link', 'cook', 'web'],
#     'red': ['cat', 'button', 'pipe', 'pants', 'mount', 'sleep', 'stick', 'file', 'worm'],
#     'assassin': ['doctor']
# }

We can use gensim to find the 10 words most related to "ambulance" in this word2vec model.

In [56]:
board = {"blue":["soul","opera","pin","space","battery","tail","draft","grass"],"red":["compound","beat","dragon","spell","face","day","Tokyo","pan","dance"],"assassin":["kangaroo"],"neutral":["luck","straw","glass","ghost","Saturn","undertaker","marble"]}

In [5]:
# model.similar_by_word('ambulance', topn=10)

C:\Users\heidi\Anaconda2\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(u'paramedics', 0.7590752243995667),
 (u'ambulances', 0.7493595480918884),
 (u'Ambulance', 0.7236292362213135),
 (u'paramedic', 0.662133514881134),
 (u'Ambulance_paramedics', 0.6315338611602783),
 (u'Ambulances', 0.6211477518081665),
 (u'LifeFlight_helicopter', 0.6147335171699524),
 (u'hospital', 0.6099206209182739),
 (u'Paramedics', 0.6081751585006714),
 (u'Ambulance_Service', 0.6080097556114197)]

Each line is the word, followed by how similar the word is to "ambulance." Some of these words word be useful, "parametics" for instance, but many are just other forms of the word "ambulance."

gensim allows us to directly find words the most similar to a whole group of words at one time.

As we can see, it produces a lot of nonsense words. We can use `restrict_vocab` to limit results to only the top n most common words in the corpus.

In [6]:
model.most_similar(
    positive=board['blue'],
    restrict_vocab=50000,
    topn=20
)

[(u'For_Restrictions', 0.43488097190856934),
 (u'bed', 0.39588358998298645),
 (u'links', 0.38411831855773926),
 (u'hook', 0.38367366790771484),
 (u'paramedics', 0.38072746992111206),
 (u'emergency', 0.37950167059898376),
 (u'jail', 0.3759669065475464),
 (u'log', 0.37062549591064453),
 (u'intensive_care', 0.3661930561065674),
 (u'call', 0.36543411016464233),
 (u'webpage', 0.3649423122406006),
 (u'tow_truck', 0.3592333197593689),
 (u'click', 0.35906946659088135),
 (u'cooked', 0.3552851676940918),
 (u'care', 0.3537469208240509),
 (u'handcuff', 0.35027384757995605),
 (u'then', 0.34921103715896606),
 (u'stay', 0.3478427529335022),
 (u'turn', 0.34607696533203125),
 (u'bookmark', 0.3458564579486847)]

This looks much better, and produces some decent clues.  
* "bed", "paramedics", "emergency" all relate to "ambulance" and "hospital." 
* "jail" could relate to "lock" and "charge." 
* "click" to "web" and "link."

But “bed” would also relate to the other team’s word “sleep”; and “click” with “button.” It would be bad to give clues which could point to the opponent’s cards. 

gensim allows for negative examples to be included as well to help avoid that.

In [57]:
neg = board['red']
neg += board['neutral']
neg += board['assassin']

model.most_similar(
    positive=board['blue'],
    negative=neg,
    restrict_vocab=50000
)

[(u'easements', 0.18227766454219818),
 (u'easement', 0.17233158648014069),
 (u'unrestricted', 0.16916465759277344),
 (u'amplifier', 0.16338896751403809),
 (u'IEEE', 0.16030414402484894),
 (u'RF', 0.15936850011348724),
 (u'MHz', 0.15754832327365875),
 (u'Scouting', 0.15407733619213104),
 (u'Draft', 0.1501276195049286),
 (u'DoD', 0.1476529985666275)]

I really like the clue "telemedicine." It's non-obvious, but relates to four words: "web," "link," "ambulance" and "hospital." This shows the potential for this method to produce novel clues.

Let's say that the clue were "telemedicine" and the four words were removed from the board, then the next team got a turn.  What might their clues be?

In [8]:
board = {
    'blue': ['spell', 'lock', 'charge', 'tail', 'link'],
    'red': ['cat', 'button', 'pipe', 'pants', 'mount', 'sleep', 'stick', 'file', 'worm'],
    'assassin': 'doctor'
}

model.most_similar(
    positive=board['red'],
    negative=board['blue'],
    restrict_vocab=50000
)

[(u'pillow', 0.43686941266059875),
 (u'bra', 0.3842337727546692),
 (u'couch', 0.38342970609664917),
 (u'tub', 0.37922778725624084),
 (u'closet', 0.36959999799728394),
 (u'sofa', 0.36713898181915283),
 (u'bathroom', 0.366258829832077),
 (u'bed', 0.36348700523376465),
 (u'crotch', 0.36245280504226685),
 (u'spoon', 0.36179912090301514)]

This appears much less successful.  The top words mostly just seem to relate to a singe word:
* pillow -> sleep
* bra -> pants
* couch -> sleep? cat?

In [12]:
wordlist = open('../../assets/word_list.txt')
words = wordlist.read().split()
print words

['Africa', 'agent', 'air', 'alien', 'alps', 'Amazon', 'ambulance', 'America', 'angel', 'Antarctica', 'apple', 'arm', 'Atlantis', 'Australia', 'Aztec', 'back', 'ball', 'band', 'bank', 'bar', 'bark', 'bat', 'battery', 'beach', 'bear', 'beat', 'bed', 'Beijing', 'bell', 'belt', 'Berlin', 'Bermuda', 'berry', 'bill', 'block', 'board', 'bolt', 'bomb', 'bond', 'boom', 'boot', 'bottle', 'bow', 'box', 'bridge', 'brush', 'buck', 'buffalo', 'bug', 'bugle', 'button', 'calf', 'Canada', 'cap', 'capital', 'car', 'card', 'carrot', 'casino', 'cast', 'cat', 'cell', 'centaur', 'center', 'chair', 'change', 'charge', 'check', 'chest', 'chick', 'China', 'chocolate', 'church', 'circle', 'cliff', 'cloak', 'club', 'code', 'cold', 'comic', 'compound', 'concert', 'conductor', 'contract', 'cook', 'copper', 'cotton', 'court', 'cover', 'crane', 'crash', 'cricket', 'cross', 'crown', 'cycle', 'Czech', 'dance', 'date', 'day', 'death', 'deck', 'degree', 'diamond', 'dice', 'dinosaur', 'disease', 'doctor', 'dog', 'draft',

In [25]:
wordlist = open('../../assets/word_list.txt')
words = wordlist.read().split("\n")
print words
for word in words:
    if len(model.wv[word]) == 0:
        print word

['Africa', 'agent', 'air', 'alien', 'alps', 'Amazon', 'ambulance', 'America', 'angel', 'Antarctica', 'apple', 'arm', 'Atlantis', 'Australia', 'Aztec', 'back', 'ball', 'band', 'bank', 'bar', 'bark', 'bat', 'battery', 'beach', 'bear', 'beat', 'bed', 'Beijing', 'bell', 'belt', 'Berlin', 'Bermuda', 'berry', 'bill', 'block', 'board', 'bolt', 'bomb', 'bond', 'boom', 'boot', 'bottle', 'bow', 'box', 'bridge', 'brush', 'buck', 'buffalo', 'bug', 'bugle', 'button', 'calf', 'Canada', 'cap', 'capital', 'car', 'card', 'carrot', 'casino', 'cast', 'cat', 'cell', 'centaur', 'center', 'chair', 'change', 'charge', 'check', 'chest', 'chick', 'China', 'chocolate', 'church', 'circle', 'cliff', 'cloak', 'club', 'code', 'cold', 'comic', 'compound', 'concert', 'conductor', 'contract', 'cook', 'copper', 'cotton', 'court', 'cover', 'crane', 'crash', 'cricket', 'cross', 'crown', 'cycle', 'Czech', 'dance', 'date', 'day', 'death', 'deck', 'degree', 'diamond', 'dice', 'dinosaur', 'disease', 'doctor', 'dog', 'draft',

C:\Users\heidi\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [29]:
model.save("../../assets/word2vec.model")

In [31]:
model = gensim.models.Word2Vec.load("../../assets/word2vec.model")

AttributeError: 'module' object has no attribute 'Word2VecKeyedVectors'